In [1]:
import numpy as np
# %matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [3]:
torch.__version__  # should be 0.4.1

'2.4.1+cu124'

In [4]:
import torchvision
torchvision.__version__  # should be 0.2.1

'0.19.1+cu124'

In [5]:
torch.cuda.is_available()

True

In [6]:
# Kaggle Kernel-dependent
input_path = "./datasets/"

### 2. Create PyTorch data generators

In [7]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'val':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train':
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'val':
    datasets.ImageFolder(input_path + 'val', data_transforms['val'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'val':
    torch.utils.data.DataLoader(image_datasets['val'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}

### 3. Create the network

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
# model = models.resnet50(pretrained=True).to(device)

# for param in model.parameters():
#     param.requires_grad = False

# model.fc = nn.Sequential(
#                nn.Linear(2048, 128),
#                nn.ReLU(inplace=True),
#                nn.Linear(128, 2)).to(device)

model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(128, 2)
).to(device)

for param in model.fc.parameters():
    param.requires_grad = True



c:\Users\sirip\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sirip\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr = 0.001)

### 4. Train the model

In [11]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

There is some error (even though the same version work on my own computer):

> RuntimeError: DataLoader worker (pid 56) is killed by signal: Bus error. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.
> RuntimeError: DataLoader worker (pid 59) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

See [this issue](https://github.com/pytorch/pytorch/issues/5301) and [that thread](https://discuss.pytorch.org/t/dataloader-randomly-crashes-after-few-epochs/20433/2). Setting `num_workers=0` in `DataLoader` solved it.

In [12]:
model_trained = train_model(model, criterion, optimizer, num_epochs=10)

Epoch 1/10
----------
train loss: 0.1814, acc: 0.9251
val loss: 0.0485, acc: 0.9823
Epoch 2/10
----------
train loss: 0.1056, acc: 0.9601
val loss: 0.0431, acc: 0.9841
Epoch 3/10
----------
train loss: 0.0847, acc: 0.9689
val loss: 0.0292, acc: 0.9903
Epoch 4/10
----------
train loss: 0.0729, acc: 0.9727
val loss: 0.0208, acc: 0.9938
Epoch 5/10
----------
train loss: 0.0728, acc: 0.9742
val loss: 0.0152, acc: 0.9947
Epoch 6/10
----------
train loss: 0.0753, acc: 0.9726
val loss: 0.0301, acc: 0.9903
Epoch 7/10
----------
train loss: 0.0678, acc: 0.9748
val loss: 0.0189, acc: 0.9947
Epoch 8/10
----------
train loss: 0.0590, acc: 0.9779
val loss: 0.0163, acc: 0.9965
Epoch 9/10
----------
train loss: 0.0604, acc: 0.9782
val loss: 0.0125, acc: 0.9947
Epoch 10/10
----------
train loss: 0.0501, acc: 0.9820
val loss: 0.0129, acc: 0.9938


In [13]:
print(model_trained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
torch.save(model_trained.state_dict(), 'models/weightsV4.h5')

In [15]:
model.load_state_dict(torch.load('models/weightsV3.h5'), strict=False)

C:\Users\sirip\AppData\Local\Temp\ipykernel_12992\2677868803.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/weightsV3.h5'), str

<All keys matched successfully>

### 5. Save and load the model

In [ ]:
model = models.resnet50(pretrained=None).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))

C:\Users\PlsDontBroken\anaconda3\envs\yolov5\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\PlsDontBroken\anaconda3\envs\yolov5\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

### 6. Make predictions on sample test images

In [ ]:
validation_img_paths = ["test/non-target/cassava_healthy_118.jpg",
                                   "test/target/Collecto_733.jpg",
                                   "test/non-target/image_809.jpg",
                                    "test/target/healthy_959.jpg",
                                    "test/target/other_diseases_360.jpg"]

img_list = [Image.open(input_path + img_path) for img_path in validation_img_paths]

In [ ]:
# transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),normalize])

In [ ]:
validation_batch = torch.stack([data_transforms['val'](img).to(device)
                                for img in img_list])

In [ ]:
pred_logits_tensor = model(validation_batch)
pred_logits_tensor

tensor([[ 2.7856, -1.4965],
        [-2.6797,  3.1765],
        [ 7.6006, -5.6845],
        [-5.7725,  5.8981],
        [-2.0434,  2.4601]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
pred_probs = F.softmax(pred_logits_tensor , dim=1).cpu().data.numpy()
pred_probs

array([[9.8637474e-01, 1.3625237e-02],
       [2.8539165e-03, 9.9714607e-01],
       [9.9999833e-01, 1.6996580e-06],
       [8.5411875e-06, 9.9999142e-01],
       [1.0949642e-02, 9.8905039e-01]], dtype=float32)

In [ ]:
ans = np.argmax(pred_probs, axis=1)
ans

array([0, 1, 0, 1, 1], dtype=int64)

In [ ]:
validation_img_paths = "test/non-target/cassava_healthy_118.jpg"

img_list = Image.open(input_path + validation_img_paths)

In [ ]:
validation_batch=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),normalize])(img_list).to(device)

In [ ]:
pred_logits_tensor = model(validation_batch.unsqueeze(0))

In [ ]:
pred_probs = F.softmax(pred_logits_tensor , dim=1).cpu().data.numpy()

In [ ]:
pred_probs

array([[0.07103631, 0.92896366]], dtype=float32)